In [1]:
import torch
import os
import pickle
from tqdm import tqdm
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
class LexicalAndVideoClassifier(torch.nn.Module):
    def __init__(self, embedding_size):
        super(LexicalAndVideoClassifier, self).__init__()
        self.linear1 = torch.nn.Linear(embedding_size, 16)
        self.linear2 = torch.nn.Linear(16, 2)
        torch.nn.init.kaiming_uniform_(self.linear1.weight)
        torch.nn.init.xavier_uniform_(self.linear2.weight)
    
    def forward(self, x):
        x = self.linear1(torch.nn.functional.relu(x))
        return self.linear2(torch.nn.functional.relu(x))

In [3]:
class LexicalAndVideoDataset(Dataset):
    def __init__(self, lexical_data, video_data, labels, lexical_file, video_file, pca_l, sc_l, pca_v, sc_v, test):
        self.lexical_embeddings = []
        self.labels = []
        with open(lexical_file, 'rb') as f:
            self.embedding_map = pickle.load(f)
        for file in lexical_data:
            embedding = self.embedding_map[file.split('.')[0]]
            if type(embedding) == np.ndarray:
                self.lexical_embeddings.append(embedding)
            else:
                self.lexical_embeddings.append(embedding.numpy())
        self.lexical_embeddings = np.array(self.lexical_embeddings)
        with open(video_file, 'rb') as f:
            embeddings_map = pickle.load(f)
        self.video_embeddings = np.array([embeddings_map[file.split('.')[0]].numpy() for file in video_data])
        if not test:
            self.lexical_embeddings = sc_l.fit_transform(self.lexical_embeddings)
            self.lexical_embeddings = pca_l.fit_transform(self.lexical_embeddings)
            self.video_embeddings = sc_v.fit_transform(self.video_embeddings)
            self.video_embeddings = pca_v.fit_transform(self.video_embeddings)
        else:
            self.lexical_embeddings = sc_l.transform(self.lexical_embeddings)
            self.lexical_embeddings = pca_l.transform(self.lexical_embeddings)
            self.video_embeddings = sc_v.transform(self.video_embeddings)
            self.video_embeddings = pca_v.transform(self.video_embeddings)
        self.lexical_embeddings = torch.from_numpy(self.lexical_embeddings).float()
        self.video_embeddings = torch.from_numpy(self.video_embeddings).float()
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        return torch.concat((self.lexical_embeddings[index], self.video_embeddings[index])), self.labels[index]

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
def train(model, train_loader, criterion, optimizer, num_epochs):
    for _ in range(num_epochs):
        model.train()
        for x, y in train_loader:
            x = x.to(device)
            y = y.to(device)
            out = model(x)
            loss = criterion(out, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

In [6]:
def eval(model, val_loader):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for x, y in val_loader:
            x = x.to(device)
            y = y.to(device)
            out = model(x)
            _, predicted = torch.max(out.data, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()
        return correct / total

In [7]:
def get_lexical_data(TRUTH_DIR, DECEPTION_DIR):
    data = []
    for file in os.listdir(TRUTH_DIR):
        data.append(file)
    for file in os.listdir(DECEPTION_DIR):
        data.append(file)
    return data

In [32]:
train_video_data = []
train_labels = []
for file in os.listdir('../../data/Clips/Truthful/'):
    train_video_data.append(file.split('.')[0])
    train_labels.append(0)
for file in os.listdir('../../data/Clips/Deceptive/'):
    train_video_data.append(file.split('.')[0])
    train_labels.append(1)

test_video_data = []
test_labels = []
for file in os.listdir('../../Dataset2/Videos/Truth/'):
    key = file.split('.')[0]
    key = '-'.join(key.split('-')[:3])
    test_video_data.append(key)
    test_labels.append(0)
for file in os.listdir('../../Dataset2/Videos/Lie/'):
    key = file.split('.')[0]
    key = '-'.join(key.split('-')[:3])
    test_video_data.append(key)
    test_labels.append(1)

train_lexical_data = get_lexical_data('../../data/Transcription/Truthful/', '../../data/Transcription/Deceptive/')
test_lexical_data = get_lexical_data('../../Dataset2/Text/Truth/', '../../Dataset2/Text/Lie/')

In [33]:
def get_accuracy(train_lexical_file, train_acoustic_file, test_lexical_file, test_acoustic_file, embedding_size, train_batch_size, num_epochs):
    sc_v = StandardScaler()
    pca_v = PCA(n_components=embedding_size // 2)
    sc_l = StandardScaler()
    pca_l = PCA(n_components=embedding_size // 2)
    train_dataset = LexicalAndVideoDataset(train_lexical_data, train_video_data, train_labels, train_lexical_file, train_acoustic_file, pca_l, sc_l, pca_v, sc_v, False)
    test_dataset = LexicalAndVideoDataset(test_lexical_data, test_video_data, test_labels, test_lexical_file, test_acoustic_file, pca_l, sc_l, pca_v, sc_v, True)
    model = LexicalAndVideoClassifier(embedding_size).to(device)
    train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    learning_rate = 1e-3
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    train(model, train_loader, criterion, optimizer, num_epochs)
    return eval(model, test_loader)

In [34]:
# Model accuracy for bert-base-uncased and resnet-3d-18
get_accuracy('../../embeddings/transcript_features_1.pkl', '../../embeddings/visual_features.pkl', '../../embeddings2/transcript_features_1.pkl', '../../embeddings2/visual_features.pkl', 64, 4, 20)

0.4697986577181208

In [35]:
# Model accuracy for bert-base-uncased and resnet-3d-18
get_accuracy('../../embeddings/transcript_features_1.pkl', '../../embeddings/visual_features.pkl', '../../embeddings2/transcript_features_1.pkl', '../../embeddings2/visual_features.pkl', 96, 4, 20)

0.46308724832214765

In [36]:
# Model accuracy for bert-base-cased-finetuned-emotion and resnet-3d-18
get_accuracy('../../embeddings/transcript_features_2.pkl', '../../embeddings/visual_features.pkl', '../../embeddings2/transcript_features_2.pkl', '../../embeddings2/visual_features.pkl', 96, 4, 40)

0.5033557046979866

In [37]:
# Model accuracy for bert-base-cased-finetuned-emotion and resnet-3d-18
get_accuracy('../../embeddings/transcript_features_2.pkl', '../../embeddings/visual_features.pkl', '../../embeddings2/transcript_features_2.pkl', '../../embeddings2/visual_features.pkl', 64, 4, 40)

0.5100671140939598

In [38]:
# Model accuracy for transformersbook/distilbert-base-uncased-finetuned-emotion and resnet-3d-18
get_accuracy('../../embeddings/transcript_features_3.pkl', '../../embeddings/visual_features.pkl', '../../embeddings2/transcript_features_3.pkl', '../../embeddings2/visual_features.pkl', 64, 4, 20)

0.5369127516778524

In [39]:
# Model accuracy for mpNet and resnet-3d-18
get_accuracy('../../embeddings/transcript_features_6.pkl', '../../embeddings/visual_features.pkl', '../../embeddings2/transcript_features_6.pkl', '../../embeddings2/visual_features.pkl', 64, 4, 20)

0.5234899328859061

In [40]:
# Model accuracy for mpNet and resnet-3d-18
get_accuracy('../../embeddings/transcript_features_6.pkl', '../../embeddings/visual_features.pkl', '../../embeddings2/transcript_features_6.pkl', '../../embeddings2/visual_features.pkl', 96, 4, 20)

0.4966442953020134

In [41]:
# Model accuracy for mpNet and resnet-3d-18
get_accuracy('../../embeddings/transcript_features_6.pkl', '../../embeddings/visual_features.pkl', '../../embeddings2/transcript_features_6.pkl', '../../embeddings2/visual_features.pkl', 96, 4, 40)

0.5100671140939598